## Matrix multiplication in PyOpenCL

In this example we will learn how to implement a simple matrix multiplication in OpenCL. Please note that there are good libraries available to handle this, such as [MAGMA](http://icl.cs.utk.edu/magma/software/) or [clBLAS](https://github.com/clMathLibraries/clBLAS). In practice, we should always use well-tested standard solutions if available. The example presented below has been adapted from https://cnugteren.github.io/tutorial/pages/page1.html.

As usual, we should first install an OpenCL environment.

We can now import pyopencl. We also need Numpy.

In [1]:
import pyopencl as cl
import numpy as np

We want to multiply two matrices of dimension 2048. For a first implementation, we assume both matrices to be square and of fixed size. The following command creates two random matrices.

In [2]:
n = 2048
dtype='float32'
A = np.random.randn(n, n).astype(dtype)
B = np.random.randn(n, n).astype(dtype)

Let us load these arrays onto the device and create space for the result output.

In [3]:
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)
mf = cl.mem_flags
a_device = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=A)
b_device = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=B)
c_device = cl.Buffer(ctx, mf.WRITE_ONLY, A.nbytes)

First, we want to measure the CPU time using the Numpy product. For timing comparisons we first define our usual Timer class again.

In [4]:
import time

class Timer:    
    def __enter__(self):
        self.start = time.time()
        return self

    def __exit__(self, *args):
        self.end = time.time()
        self.interval = self.end - self.start

In [5]:
with Timer() as t:
    C = A.dot(B)
print(t.interval)

0.20166397094726562


Remember how a matrix multiplication works. We have a triple for loop of the form

    for i in range(n):
        for j in range(n):
            for k in range(n):
                C[i, j] += A[i, k] * B[k, j]
   
The most trivial way to parallelize the execution is to handle the two outer loops in parallel and have the inner loop be handled by the kernel.

In [6]:
kernel = """
__kernel void simple_matmul(const int N,
                          const __global float* A,
                          const __global float* B,
                          __global float* C){
        // Thread identifiers
        const int globalRow = get_global_id(0);
        const int globalCol = get_global_id(1);
        // Compute a single element (loop over K)
        float acc = 0.0f; 
        for (int k=0; k<N; k++)
            acc += A[N * globalRow + k] * B[N * k + globalCol];
        // Store the result
        C[globalRow*N + globalCol] = acc;
        }
"""

In [7]:
prg = cl.Program(ctx, kernel).build()
simple_matmul = prg.simple_matmul

Let's run the kernel.

In [ ]:
with Timer() as t:
    event = simple_matmul(queue, (n, n), (1, 1), np.int32(n), a_device, b_device, c_device)
    event.wait()
print(t.interval)

The execution is quite slow. One issue is that we have a lot of load operations from the global memory compared to the computations. This is not an issue on machines with large and sophisticated caches (such as an Intel Xeon). The following kernel proceeds by creating work groups and assigning subblocks to each workgroup.

In the following we define the local work group size through the C macro *TS*.

In [8]:
kernel = """
__kernel void blocked_matmul(const int N,
                      const __global float* A,
                      const __global float* B,
                      __global float* C) {
                      
    // Thread identifiers
    const int row = get_local_id(0); // Local row ID (max: TS)
    const int col = get_local_id(1); // Local col ID (max: TS)
    const int globalRow = TS*get_group_id(0) + row; // Row ID of C (0..M)
    const int globalCol = TS*get_group_id(1) + col; // Col ID of C (0..N)
 
    // Local memory to fit a tile of TS*TS elements of A and B
    __local float Asub[TS][TS];
    __local float Bsub[TS][TS];
 
    // Initialise the accumulation register
    float acc = 0.0f;
    
    // Loop over all tiles
    const int numTiles = N/TS;
    for (int t=0; t<numTiles; t++) {{
 
        // Load one tile of A and B into local memory
        const int tiledRow = TS*t + row;
        const int tiledCol = TS*t + col;
        Asub[row][col] = A[tiledCol + N * globalRow];
        Bsub[row][col] = B[globalCol + tiledRow * N];
 
        // Synchronise to make sure the tile is loaded
        barrier(CLK_LOCAL_MEM_FENCE);
 
        // Perform the computation for a single tile
        for (int k=0; k<TS; k++) {{
            acc += Asub[row][k] * Bsub[k][col];
        }}
 
        // Synchronise before loading the next tile
        barrier(CLK_LOCAL_MEM_FENCE);
    }}
 
    // Store the final result in C
    C[globalCol + globalRow * N] = acc;
}
"""

In [9]:
ts = 4
prg = cl.Program(ctx, kernel).build(options=['-D', 'TS={0}'.format(ts)])
blocked_matmul = prg.blocked_matmul

In [12]:
with Timer() as t:
    event = blocked_matmul(queue, (n, n), (ts, ts), np.int32(n), a_device, b_device, c_device)
    event.wait()
print(t.interval)

2.015583038330078


It is now easy to experiment with different work group sizes. Optimal work group sizes depend on the details of the underlying architecture.

## Exercise

OpenCL nows vector variables of the type `float2`, `float4`, etc. These store multiple floats at the same time and can often be better optimized by the underlying runtime system for the in-built SIMD architecture. Check out the OpenCL standard to learn about these vector types and try to use them in the kernel.